In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.

---

신경망 구축하기
========================

신경망은 데이터에 대해 연산을 수행하는 계층/모듈로 구성됩니다. [torch.nn](https://pytorch.org/docs/stable/nn.html) 네임스페이스는 자신만의 신경망을 구축하는 데 필요한 모든 구성 요소를 제공합니다. PyTorch의 모든 모듈은 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)을 상속받습니다. 신경망 자체도 다른 모듈(계층)로 구성된 하나의 모듈입니다. 이러한 중첩 구조를 통해 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

다음 섹션에서는 FashionMNIST 데이터셋의 이미지를 분류하기 위한 신경망을 구축해 보겠습니다.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on a hardware accelerator like the
GPU or MPS, if available. Let\'s check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) or
[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are
available, otherwise we use the CPU.

---

훈련을 위한 장치 얻기
=======================

가능하다면 GPU나 MPS와 같은 하드웨어 가속기에서 모델을 훈련시키고자 합니다. [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) 또는 [torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html)를 사용할 수 있는지 확인해 보겠습니다. 사용할 수 없다면 CPU를 사용합니다.


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.

---

클래스 정의하기
================

우리는 `nn.Module`을 상속받아 신경망을 정의하고, `__init__` 메서드에서 신경망 계층을 초기화합니다. 모든 `nn.Module` 하위 클래스는 `forward` 메서드에서 입력 데이터에 대한 연산을 구현합니다.


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
class NeuralNetwork(nn.Module): # nn.Module 상속받아 PyTorch 모듈 생성

    # 초기화 메소드
    def __init__(self):
        super().__init__() # 부모 클래스인 nn.Module의 초기화 메서드를 호출
        self.flatten = nn.Flatten() # 입력 데이터를 1차원으로 평탕화하는 계층을 정의
        self.linear_relu_stack = nn.Sequential( # nn.Sequential을 사용하여 여러 계층을 순차적으로 쌓음
            nn.Linear(28*28, 512), # (input: 28*28(784), output: 512)
            nn.ReLU(),
            nn.Linear(512, 512), # (input: 512, output: 512)
            nn.ReLU(),
            nn.Linear(512, 10), # (input: 512, output: 10)
        )

    # Forward Pass
    def forward(self, x):
        x = self.flatten(x) # 입력을 1차원으로 평탄화
        logits = self.linear_relu_stack(x) # 초기화 메소드에서 정의한 여러 계층 통과
        return logits # 최종 출력 logits 반환

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.

---

`NeuralNetwork`의 인스턴스를 생성하고, 이를 `device`로 이동시킨 후, 그 구조를 출력합니다.

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.

---

모델을 사용하기 위해서는 입력 데이터를 모델에 전달합니다. 이는 모델의 `forward` 메서드를 실행하며, 동시에 일부 [백그라운드 작업](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)도 수행합니다. `model.forward()` 메서드를 직접 호출하지 마세요!

입력에 대해 모델을 호출하면 2차원 텐서가 반환됩니다. 이 텐서의 dim=0은 10개 클래스 각각에 대한 원시 예측값의 출력에 해당하고, dim=1은 각 출력의 개별 값에 해당합니다. 예측 확률을 얻기 위해서는 이 결과를 `nn.Softmax` 모듈의 인스턴스에 통과시킵니다.

In [17]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='mps:0')


In [29]:
# 28 * 28 크기의 랜덤한 입력 이미지 생성
# device에 지정된 장치(여기선 MPS)에 텐서를 생성
X = torch.rand(1, 28, 28, device=device) 

logits = model(X) # 생성된 입력 X를 모델에 전달하여 예측 수행

# logits에 softmax 함수를 적용하여 확률 분포로 변환
# dim = 1은 softmax를 클래스(레이블)차원에 대해 적용함을 의미
pred_probab = nn.Softmax(dim=1)(logits)

y_pred = pred_probab.argmax(1) # pred_probab에서 가장 높은 확률을 가진 클래스의 인덱스를 찾음
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='mps:0')


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.

---

모델 계층
============

FashionMNIST 모델의 계층을 자세히 살펴보겠습니다. 이를 설명하기 위해, 28x28 크기의 3개 이미지로 구성된 샘플 미니배치를 사용하여 네트워크를 통과시키면서 어떤 일이 일어나는지 살펴보겠습니다.

In [30]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).

---

nn.Flatten
==========

[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) 계층을 초기화하여 각 2D 28x28 이미지를 784개의 연속된 픽셀 값 배열로 변환합니다 (미니배치 차원(dim=0)은 유지됩니다).

In [31]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.

---

nn.Linear
=========

[선형 계층](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)은 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈입니다.


In [32]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.

---

nn.ReLU
=======

비선형 활성화 함수는 모델의 입력과 출력 사이에 복잡한 매핑을 생성합니다. 이들은 선형 변환 후에 적용되어 *비선형성*을 도입하며, 이를 통해 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

이 모델에서는 선형 계층 사이에 [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)를 사용하지만, 모델에 비선형성을 도입하기 위한 다른 활성화 함수들도 있습니다.


In [33]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1338,  0.0409, -0.2365,  0.2345,  0.1725, -0.1421,  0.3883,  0.5868,
          0.2607, -0.4720,  0.0132, -0.2630,  0.2821,  0.2940, -0.0622, -0.1723,
         -0.4742, -0.4326,  0.1610, -0.3325],
        [ 0.0524,  0.3491, -0.1158,  0.2554,  0.2242,  0.0698, -0.1217,  0.2525,
          0.4569, -0.5268,  0.2112, -0.3254, -0.0820,  0.5374,  0.0272,  0.2705,
         -0.2851, -0.1652,  0.5313, -0.3906],
        [ 0.0700, -0.0046, -0.0320,  0.3942,  0.0094,  0.2909,  0.0312,  0.5041,
          0.3913, -0.7047, -0.2276,  0.0432, -0.0324,  0.7135,  0.4199, -0.0696,
         -0.3654,  0.2630,  0.2945, -0.3621]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0409, 0.0000, 0.2345, 0.1725, 0.0000, 0.3883, 0.5868, 0.2607,
         0.0000, 0.0132, 0.0000, 0.2821, 0.2940, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1610, 0.0000],
        [0.0524, 0.3491, 0.0000, 0.2554, 0.2242, 0.0698, 0.0000, 0.2525, 0.4569,
         0.0000, 0.2112, 0.0000, 0.0000, 0.5374, 0.02

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.

---

 nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)은 모듈들의 순서가 있는 컨테이너입니다. 데이터는 정의된 순서대로 모든 모듈을 통과합니다. 순차적 컨테이너를 사용하여 `seq_modules`와 같은 간단한 네트워크를 빠르게 구성할 수 있습니다.

In [34]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.

nn.Softmax
==========

신경망의 마지막 선형 계층은 [-무한대, 무한대] 범위의 원시 값인 [logits]을 반환합니다. 이 값들은 [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) 모듈에 전달됩니다. logits는 [0, 1] 범위의 값으로 스케일링되어 각 클래스에 대한 모델의 예측 확률을 나타냅니다. `dim` 매개변수는 값들의 합이 1이 되어야 하는 차원을 지정합니다.


In [35]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.

---

모델 매개변수
================

신경망 내부의 많은 계층들은 *매개변수화*되어 있습니다. 즉, 훈련 중에 최적화되는 관련 가중치와 편향을 가지고 있습니다. `nn.Module`을 상속받으면 모델 객체 내부에 정의된 모든 필드를 자동으로 추적하며, 모델의 `parameters()` 또는 `named_parameters()` 메서드를 사용하여 모든 매개변수에 접근할 수 있게 합니다.

이 예제에서는 각 매개변수를 반복하면서 그 크기와 값의 미리보기를 출력합니다.

In [36]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 3.1542e-02, -9.5490e-03,  1.9903e-02,  ...,  3.4612e-02,
          1.6581e-02,  1.1486e-02],
        [ 3.2123e-02, -9.1787e-05, -5.9802e-03,  ...,  1.1902e-02,
         -1.0127e-02,  3.4876e-02]], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0305,  0.0341], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0216,  0.0189,  0.0102,  ..., -0.0342,  0.0138, -0.0049],
        [ 0.0006, -0.0420, -0.0409,  ..., -0.0354

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
